In [7]:

import pandas as pd

def find_best_row(group):
    min_all = group.loc[(group['test_mse'] == group['test_mse'].min()) & 
                        (group['test_smape'] == group['test_smape'].min()) & 
                        (group['test_mase'] == group['test_mase'].min())]

    if not min_all.empty:
        return min_all.iloc[0]
    
    for metric_combination in [('test_mse', 'test_smape'), ('test_mse', 'test_mase'), ('test_smape', 'test_mase')]:
        min_two = group.loc[(group[metric_combination[0]] == group[metric_combination[0]].min()) & 
                            (group[metric_combination[1]] == group[metric_combination[1]].min())]
        if not min_two.empty:
            return min_two.iloc[0]
        
    return group.loc[group['test_mase'] == group['test_mase'].min()].iloc[0]

base_path = f"/home/../multiTS/NFT/models/tests/test_trend/"
df = pd.read_excel(f'{base_path}evaluate_sinusodial_trend_on_syntatic_data.xlsx')

poly_true = df[df['Is Poly'] == True]
poly_false = df[df['Is Poly'] == False]

best_rows_poly_true = poly_true.groupby(['Lookback', 'Horizon', 'Vars']).apply(find_best_row).reset_index(drop=True)

In [11]:
print(df.shape)
print(poly_true.shape)
print(poly_false.shape)
print(best_rows_poly_true.shape)

(1266, 11)
(632, 11)
(634, 11)
(53, 11)


In [12]:

import pandas as pd

def find_best_row(group):
    min_all = group.loc[(group['test_mse'] == group['test_mse'].min()) & 
                        (group['test_smape'] == group['test_smape'].min()) & 
                        (group['test_mase'] == group['test_mase'].min())]

    if not min_all.empty:
        return min_all.iloc[0]
    
    for metric_combination in [('test_mse', 'test_smape'), ('test_mse', 'test_mase'), ('test_smape', 'test_mase')]:
        min_two = group.loc[(group[metric_combination[0]] == group[metric_combination[0]].min()) & 
                            (group[metric_combination[1]] == group[metric_combination[1]].min())]
        if not min_two.empty:
            return min_two.iloc[0]
        
    return group.loc[group['test_mase'] == group['test_mase'].min()].iloc[0]

base_path = f"/home/../multiTS/NFT/models/tests/test_trend/"
df = pd.read_excel(f'{base_path}evaluate_sinusodial_trend_on_syntatic_data.xlsx')

poly_true = df[df['Is Poly'] == True]
poly_false = df[df['Is Poly'] == False]

best_rows_poly_true = poly_true.groupby(['Lookback', 'Horizon', 'Vars']).apply(find_best_row).reset_index(drop=True)
best_rows_combined = pd.DataFrame()

for _, row in best_rows_poly_true.iterrows():
    matching_row = poly_false[
        (poly_false['Lookback'] == row['Lookback']) & 
        (poly_false['Horizon'] == row['Horizon']) & 
        (poly_false['Vars'] == row['Vars']) & 
        (poly_false['Epochs'] == row['Epochs']) & 
        (poly_false['Stacks'] == row['Stacks']) & 
        (poly_false['Dagree'] == row['Dagree']) & 
        (poly_false['Blocks'] == row['Blocks'])
    ]
    best_rows_combined = pd.concat([best_rows_combined, pd.DataFrame([row])], ignore_index=True)
    if not matching_row.empty:
        best_rows_combined = pd.concat([best_rows_combined, matching_row.head(1)], ignore_index=True)

if best_rows_combined.empty:
    print("No matching rows were found.")
else:
    best_rows_combined.to_excel(f'{base_path}best_rows_combined.xlsx', index=False)


In [2]:
import pandas as pd

# Load the combined Excel file
best_combined = pd.read_excel('/home/../multiTS/NFT/results/combined_data.xlsx')

# Initialize the new column with NaN (Not a Number)
best_combined['Poly_Performance'] = pd.NA

# Define the metrics you want to compare. You can add more metrics here based on your actual data.
metrics = ['test_mse', 'test_smape', 'test_mase']

# Sort the dataframe for correct row comparisons
best_combined.sort_values(by=['Lookback', 'Horizon'], inplace=True)

# Go over each two rows and compare their metrics
for i in range(0, len(best_combined) - 1, 2):  # Step by 2 since we're comparing pairs
    row_1 = best_combined.iloc[i]
    row_2 = best_combined.iloc[i + 1]

    # Assuming row_1 is Poly=True and row_2 is Poly=False because of sorting, if not, check and swap
    if not row_1['Is Poly']:
        row_1, row_2 = row_2, row_1  # Swap if the first row is not Poly=True
    
    # Compare the metrics
    if all(row_1[metric] <= row_2[metric] for metric in metrics):
        # If all metrics of row_1 (Poly=True) are better or equal
        best_combined.at[i, 'Poly_Performance'] = 1
        best_combined.at[i + 1, 'Poly_Performance'] = 0
    elif all(row_2[metric] <= row_1[metric] for metric in metrics):
        # If all metrics of row_2 (Poly=False) are better or equal
        best_combined.at[i, 'Poly_Performance'] = 0
        best_combined.at[i + 1, 'Poly_Performance'] = -1

# Save the updated DataFrame back to Excel
best_combined.to_excel('/home/../multiTS/NFT/models/tests/test_trend/best_rows_combined_updated.xlsx', index=False)


In [7]:
import pandas as pd

# Load the combined Excel file
best_combined = pd.read_excel('/home/../multiTS/NFT/results/combined_data.xlsx')

# Initialize the new column with NaN (Not a Number)
best_combined['Poly_Performance'] = pd.NA

# Define the metrics you want to compare.
metrics = ['test_mse', 'test_smape', 'test_mase']

# Sort the dataframe for correct row comparisons
best_combined.sort_values(by=['Lookback', 'Horizon'], inplace=True)

# Initialize an empty list to keep track of rows to remove
rows_to_remove = []

# Go over each two rows and compare their metrics
for i in range(0, len(best_combined) - 1, 2):  # Step by 2 since we're comparing pairs
    row_1 = best_combined.iloc[i]
    row_2 = best_combined.iloc[i + 1]

    # Assuming row_1 is Poly=True and row_2 is Poly=False because of sorting, if not, check and swap
    if not row_1['Is Poly']:
        row_1, row_2 = row_2, row_1  # Swap if the first row is not Poly=True
    
    # Compare the metrics
    better_metrics = sum(row_1[metric] <= row_2[metric] for metric in metrics)
    if better_metrics >= 2:  # If at least two metrics are better or equal for Poly=True
        best_combined.at[i, 'Poly_Performance'] = 1
        best_combined.at[i + 1, 'Poly_Performance'] = 0
    else:
        best_combined.at[i, 'Poly_Performance'] = 0
        best_combined.at[i + 1, 'Poly_Performance'] = -1
        # Mark these rows for removal
        rows_to_remove.extend([i, i + 1])

# # Remove the rows where Poly=True does not perform better in at least two metrics
best_combined.drop(index=rows_to_remove, inplace=True)

# Save the updated DataFrame back to Excel
best_combined.to_excel('/home/../multiTS/NFT/models/tests/test_trend/removed_updated.xlsx', index=False)


In [12]:
    import pandas as pd

    # Load the combined Excel file
    best_combined = pd.read_excel('/home/../multiTS/NFT/results/combined_data.xlsx')

    # Initialize the new columns with extracted substrings
    best_combined['series'] = best_combined['Data'].apply(lambda x: x[5:16])
    best_combined['year'] = best_combined['Data'].apply(lambda x: x[17:21])

    # Initialize the 'Poly_Performance' column with NaN (Not a Number)
    best_combined['Poly_Performance'] = pd.NA

    # Define the metrics you want to compare.
    metrics = ['test_mse', 'test_smape', 'test_mase']

    # Sort the dataframe for correct row comparisons
    best_combined.sort_values(by=['Lookback', 'Horizon'], inplace=True)

    # Initialize an empty list to keep track of rows to remove
    rows_to_remove = []

    # Go over each two rows and compare their metrics
    for i in range(0, len(best_combined) - 1, 2):  # Step by 2 since we're comparing pairs
        row_1 = best_combined.iloc[i]
        row_2 = best_combined.iloc[i + 1]

        # Assuming row_1 is Poly=True and row_2 is Poly=False because of sorting, if not, check and swap
        if not row_1['Is Poly']:
            row_1, row_2 = row_2, row_1  # Swap if the first row is not Poly=True
        
        # Compare the metrics
        better_metrics = sum(row_1[metric] <= row_2[metric] for metric in metrics)
        if better_metrics >= 2:  # If at least two metrics are better or equal for Poly=True
            best_combined.at[i, 'Poly_Performance'] = 1
            best_combined.at[i + 1, 'Poly_Performance'] = 0
        else:
            # Mark these rows for removal
            rows_to_remove.extend([i, i + 1])

    # Remove the rows where Poly=True does not perform better in at least two metrics
    best_combined.drop(index=rows_to_remove, inplace=True)

    # Save the updated DataFrame back to Excel
    best_combined.to_excel('/home/../multiTS/NFT/models/tests/test_trend/best_rows_combined_updated.xlsx', index=False)


In [14]:
import pandas as pd

# Load the combined Excel file
best_combined = pd.read_excel('/home/../multiTS/NFT/results/combined_data.xlsx')

# Initialize the new columns with extracted substrings
best_combined['series'] = best_combined['Data'].apply(lambda x: x[5:16])
best_combined['year'] = best_combined['Data'].apply(lambda x: x[17:21])

# Initialize the 'Poly_Performance' column with NaN (Not a Number)
best_combined['Poly_Performance'] = pd.NA

# Define the metrics you want to compare.
metrics = ['test_mse', 'test_smape', 'test_mase']

# Sort the dataframe for correct row comparisons
best_combined.sort_values(by=['Lookback', 'Horizon', 'Epochs', 'Blocks', 'Data'], inplace=True)

# Initialize an empty list to keep track of rows to remove
rows_to_remove = []

# Go over each two rows and compare their metrics
for i in range(0, len(best_combined) - 1, 2):  # Step by 2 since we're comparing pairs
    row_1 = best_combined.iloc[i]
    row_2 = best_combined.iloc[i + 1]

    # Check if the rows have the same values for specified columns and opposite values in 'Is Poly' column
    if all(row_1[col] == row_2[col] for col in ['Data', 'Lookback', 'Horizon', 'Epochs', 'Blocks']) and row_1['Is Poly'] != row_2['Is Poly']:
        # Assuming row_1 is Poly=True and row_2 is Poly=False because of sorting, if not, check and swap
        if not row_1['Is Poly']:
            row_1, row_2 = row_2, row_1  # Swap if the first row is not Poly=True
        
        # Compare the metrics
        better_metrics = sum(row_1[metric] <= row_2[metric] for metric in metrics)
        if better_metrics >= 2:  # If at least two metrics are better or equal for Poly=True
            best_combined.at[i, 'Poly_Performance'] = 1
            best_combined.at[i + 1, 'Poly_Performance'] = 0
        else:
            # Mark these rows for removal
            rows_to_remove.extend([i, i + 1])
    else:
        print('hi')
        # Mark these rows for removal since they don't match the criteria
        rows_to_remove.extend([i, i + 1])

# Remove the rows that do not match the comparison criteria
best_combined.drop(index=rows_to_remove, inplace=True)

# Save the updated DataFrame back to Excel
best_combined.to_excel('/home/../multiTS/NFT/models/tests/test_trend/best_rows_combined_updated.xlsx', index=False)


hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


In [ ]:
import pandas as pd

# Load the combined Excel file
best_combined = pd.read_excel('/home/../multiTS/NFT/results/combined_data.xlsx')

# Initialize the new columns with extracted substrings
best_combined['series'] = best_combined['Data'].apply(lambda x: x[5:16])
best_combined['year'] = best_combined['Data'].apply(lambda x: x[17:21])

# Initialize the 'Poly_Performance' column with NaN (Not a Number)
best_combined['Poly_Performance'] = pd.NA

# Define the metrics you want to compare.
metrics = ['test_mse', 'test_smape', 'test_mase']

# Create a new DataFrame for the paired rows
paired_rows = pd.DataFrame()

# Function to find matching row
def find_matching_row(current_row, all_rows):
    for i, row in all_rows.iterrows():
        # Check if the rows have the same values for specified columns and opposite values in 'Is Poly' column
        if all(current_row[col] == row[col] for col in ['Data', 'Lookback', 'Horizon', 'Epochs', 'Blocks']) and current_row['Is Poly'] != row['Is Poly']:
            return row
    return None  # Return None if no match is found

# Iterate over each row to find its pair based on specified conditions
for index, row in best_combined.iterrows():
    matched_row = find_matching_row(row, best_combined)
    if matched_row is not None:
        # Add the original row and its match to the new DataFrame
        paired_row = pd.concat([row, matched_row]).to_frame().T
        paired_rows = pd.concat([paired_rows, paired_row], ignore_index=True)

# Resetting the index for paired_rows to ensure proper alignment
paired_rows.reset_index(drop=True, inplace=True)

# Determine Poly Performance for each pair and arrange them next to each other
for i in range(0, len(paired_rows), 2):  # Process each pair
    row_1 = paired_rows.iloc[i]
    row_2 = paired_rows.iloc[i + 1]
    # Assuming row_1 is Poly=True and row_2 is Poly=False because of sorting, if not, check and swap
    if not row_1['Is Poly']:
        row_1, row_2 = row_2, row_1  # Swap if the first row is not Poly=True

    # Compare the metrics
    better_metrics = sum(row_1[metric] <= row_2[metric] for metric in metrics)
    if better_metrics >= 2:  # If at least two metrics are better or equal for Poly=True
        paired_rows.at[i, 'Poly_Performance'] = 1
        paired_rows.at[i + 1, 'Poly_Performance'] = 0

# Save the paired DataFrame back to Excel
paired_rows.to_excel('/home/../multiTS/NFT/models/tests/test_trend/paired_rows_combined.xlsx', index=False)


In [18]:
import pandas as pd

# Load the combined Excel file
best_combined = pd.read_excel('/home/../multiTS/NFT/results/combined_data.xlsx')

# Initialize the new columns with extracted substrings
best_combined['series'] = best_combined['Data'].apply(lambda x: x[5:16])
best_combined['year'] = best_combined['Data'].apply(lambda x: x[17:21])

# Define the metrics you want to compare.
metrics = ['test_mse', 'test_smape', 'test_mase']

# Create a new DataFrame for the paired rows and their comparisons
paired_rows = pd.DataFrame()

# List to keep track of indices to drop from the original DataFrame
indices_to_drop = []

# Iterate over the DataFrame without modifying it while iterating
for index, row in best_combined.iterrows():
    # Skip if row already paired and marked for deletion
    if index in indices_to_drop:
        continue

    # Finding the matching row
    for match_index, match_row in best_combined.iterrows():
        # Ensure not comparing the same row and hasn't been paired already
        if index != match_index and match_index not in indices_to_drop:
            # Check for same values in specified columns and opposite in 'Is Poly'
            if all(row[col] == match_row[col] for col in ['Data', 'Lookback', 'Horizon', 'Epochs', 'Blocks']) and row['Is Poly'] != match_row['Is Poly']:
                # Add index of matching row to list for removal
                indices_to_drop.extend([index, match_index])

                # Perform metrics comparison and assign Poly Performance
                better_metrics = sum(row[metric] <= match_row[metric] for metric in metrics)
                row['Poly_Performance'] = 1 if better_metrics >= 2 else 0
                match_row['Poly_Performance'] = -1 if better_metrics < 2 else 0

                # Combine the two rows and append to the new DataFrame
                comparison_pair = pd.concat([row, match_row]).to_frame().T
                paired_rows = pd.concat([paired_rows, comparison_pair], ignore_index=True)

                # Break out of the inner loop once a match is found
                break

# Drop the paired rows from the original DataFrame
best_combined = best_combined.drop(indices_to_drop).reset_index(drop=True)

# Save the paired rows to a new Excel file
paired_rows.to_excel('/home/../multiTS/NFT/models/tests/test_trend/paired_rows.xlsx', index=False)

# Save the remaining unpaired rows back to a different Excel file
best_combined.to_excel('/home/../multiTS/NFT/models/tests/test_trend/unpaired_rows.xlsx', index=False)
